In [25]:
import requests
import pandas as pd
import numpy as np
import pyodbc
import psycopg2
from datetime import datetime
from dotenv import load_dotenv
import os


In [26]:
def connect_to_database(env_file):
    """
    Carrega variáveis de ambiente de um arquivo .env especificado e estabelece uma conexão com o banco de dados.
    
    Args:
    env_file (str): Caminho para o arquivo .env.
    
    Returns:
    connection: Objeto de conexão com o banco de dados.
    cursor: Objeto cursor para interagir com o banco de dados.
    """
    # Carregar variáveis de ambiente do arquivo .env especificado
    load_dotenv(env_file)
    
    # Parâmetros da conexão
    SERVER = os.getenv('SERVER_MIS')
    DATABASE = os.getenv('DATABASE_MIS')
    USERNAME = os.getenv('USERNAME_MIS')
    PASSWORD = os.getenv('PASSWORD_MIS')
    DRIVER = os.getenv('DRIVER_MIS')
    PORT = os.getenv('PORT_MIS') 
    
    # Verificar se todas as variáveis de ambiente necessárias foram carregadas
    if not all([SERVER, DATABASE, USERNAME, PASSWORD, DRIVER]):
        raise ValueError("Certifique-se de que todas as variáveis de ambiente (SERVER_MIS, DATABASE_MIS, USERNAME_MIS, PASSWORD_MIS, DRIVER_MIS) estejam definidas no arquivo .env.")
    
    # Conexão com o SQL Server
    connection_string = f'DRIVER={DRIVER};SERVER={SERVER},{PORT};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()
    
    return connection, cursor

"""
Exemplo de uso:

env_file_path = 'path/to/your/.env'  # Substitua pelo caminho para o seu arquivo .env

connection, cursor = connect_to_database(env_file_path)

Agora você pode usar 'connection' e 'cursor' para interagir com o banco de dados
"""

"\nExemplo de uso:\n\nenv_file_path = 'path/to/your/.env'  # Substitua pelo caminho para o seu arquivo .env\n\nconnection, cursor = connect_to_database(env_file_path)\n\nAgora você pode usar 'connection' e 'cursor' para interagir com o banco de dados\n"

In [27]:
# Carrega as variáveis de ambiente do arquivo .env
#load_dotenv(dotenv_path=r"C:\Users\Matheus Alexander\Desktop\DW\Data-Warehouse\Environment\.env")

env_file_path = r'C:\Users\int.matheus\Desktop\DW\Data-Warehouse\Environment\.env_02'

connection, cursor = connect_to_database(env_file_path)

In [22]:
data = cursor.execute("Select top 10 * from users;")

In [23]:
# Obter os dados
columns = [column[0] for column in cursor.description]  # Obter nomes das colunas
data = cursor.fetchall()  # Obter todas as linhas

# Converter os dados para uma lista de dicionários
result = [dict(zip(columns, row)) for row in data]

# Opcional: Converter os dados para um DataFrame do Pandas
df = pd.DataFrame(result)

# Exibir os resultados
display(df)




,Userld,Agentld,Flag_Agent,Flag_Supervisor,Flag_Manager,Activated,LastLogin,Email,Login,Password,UserName,EnablePersonalTransfer
0,1,1,0,0,1,1,2012-11-19 12:19:03.293,,freedom,gWhBxyJE24sy55HmanF0vw==,Freedom,0
1,52,52,1,0,0,1,NaT,,agente1,L1UcEqXSXfk=,Agente1,1
2,97,97,0,1,0,1,NaT,,supervisor,0TqRtUCJA2k=,Supervisor,0
3,1098,1098,1,0,0,1,NaT,,agente2,L1UcEqXSXfk=,Agente2,1
4,1099,1099,0,0,1,0,NaT,,guilherme_augusto,Cbstrged/DM=,GUILHERME AUGUSTO,0
5,1100,1100,0,0,1,1,NaT,,leticia_albuquerque,Cbstrged/DM=,Leticia Monteiro Albuquerque,0
6,1101,1101,0,0,1,0,NaT,,silvestre_junior,o1TLsizroW2xJKZWCZyqyA==,SILVESTRE JUNIOR,0
7,1102,1102,0,0,1,0,NaT,,monaliza_rocha,Cbstrged/DM=,MONALIZA ROCHA,0
8,1103,1103,0,1,0,0,NaT,,gleiciane_cardoso,Cbstrged/DM=,ANTONIA GLEICIANE CARDOSO DA SILVA,0
9,1104,1104,0,0,1,0,NaT,,bruno_tavares,GY1vtir07eGX4nQansR93A==,BRUNO TAVARES CHAVES DE LIMA,0


In [24]:
# Fechar a conexão
cursor.close()
connection.close()